In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
import matplotlib.pyplot as plt
import os

app = Flask(__name__)
app.secret_key = 'votre_clé_secrète'

# Mock database for demonstration purposes
users = {'user1': {'password': 'pass1', 'posts': []},
         'user2': {'password': 'pass2', 'posts': []}}

@app.route('/')
def index():
    if 'username' in session:
        return redirect(url_for('dashboard'))
    return render_template('login.html')

@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']

    if username in users and users[username]['password'] == password:
        session['username'] = username
        return redirect(url_for('dashboard'))
    else:
        return 'Invalid username or password'

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

@app.route('/dashboard')
def dashboard():
    if 'username' in session:
        username = session['username']
        user_posts = users[username]['posts']
        return render_template('dashboard.html', username=username, user_posts=user_posts)
    else:
        return redirect(url_for('index'))

@app.route('/write_post', methods=['GET', 'POST'])
def write_post():
    if 'username' in session:
        if request.method == 'POST':
            username = session['username']
            post_content = request.form['post_content']
            users[username]['posts'].append(post_content)

            # Example: Generate a simple plot using matplotlib
            post_index = len(users[username]['posts']) - 1
            plot_filename = f'{username}_plot_{post_index}.png'
            plot_path = os.path.join('static', 'plots', plot_filename)
            plt.plot([1, 2, 3, 4], [10, 20, 25, 30])
            plt.xlabel('X-axis')
            plt.ylabel('Y-axis')
            plt.savefig(plot_path)
            plt.close()

            return render_template('write_post.html', plot_path=plot_path)

        return render_template('write_post.html')
    else:
        return redirect(url_for('index'))

@app.route('/view_post/<int:post_index>')
def view_post(post_index):
    if 'username' in session:
        username = session['username']
        user_posts = users[username]['posts']

        if 0 <= post_index < len(user_posts):
            post_content = user_posts[post_index]

            # Example: Generate a simple plot using matplotlib
            plot_filename = f'{username}_plot_{post_index}.png'
            plot_path = os.path.join('static', 'plots', plot_filename)
            plt.plot([1, 2, 3, 4], [10, 20, 25, 30])
            plt.xlabel('X-axis')
            plt.ylabel('Y-axis')
            plt.savefig(plot_path)
            plt.close()

            return render_template('view_post.html', post_content=post_content, plot_path=plot_path)

    return 'Post not found or unauthorized'

if __name__ == '__main__':
    app.run(debug=True)
